<a href="https://colab.research.google.com/github/WillEversHood/LSTM-for-Stonks/blob/main/Statistical_Brakout_Trading_Strat_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a basic breakout trading strategy based on the methods originally developed by Richard Dennis and Bill Eckhart. The idea here is to use simplistic statistical tools to identify 'breakout' event and trade on them accordingly.

**1.** Data Collection
**2.** Trading Rules
**3.** Fake Trader
**4.** Parameters
**5.** Execution
**6.** Plots
**7.** Performance Analysis & Next Steps

In [ ]:
#1. collect data we will use yfinance to accomplish this we will collect data from a variety of company types and markets try to cover
# tech, industrial markets, commodoties of different types etc.
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

'''
tickers = ['AAPL', 'NVDA', 'QQQ', 'CAT', 'CAT', 'UNP', 'XLI', 'GLD', 'USO', 'GSG', 'XME']
data_list = []
for i in range(len(tickers)):
  data_list.append(yf.download(tickers[i], period='5y', interval='1d'))
'''
apple = yf.download('AAPL', period='5y', interval='1d')
apple = apple.drop(columns=['High','Low','Open', 'Volume'])
apple.head()




[*********************100%***********************]  1 of 1 completed


Price,Close
Ticker,AAPL
Date,
2020-03-12,60.240215
2020-03-13,67.457489
2020-03-16,58.779289
2020-03-17,61.363819
2020-03-18,59.861633


In [ ]:
#2. here it is iimportant to establish conservative trading rules that find a
# reasonable balance between risk and conservatism. breaking eggs sucks omelets are cool

# They recommend using an exponential weighted average to detect break outs note
# this has nothing to do with the exponential function and is a literally just a
# weighted averge prioritizing more recent values

# might in the future write my own method because span here simply is a parameter to describe the weight that each variable gets
# essentially every data point in the series still gets used.
def EMA(ticker, span):
  return ticker.ewm(span, adjust=False).mean()

def Volatility(ticker, span):
  return ticker.rolling(span).std()

# Determine Price consolidation points
def high_points(ticker, span, line):
  ticker = ticker.tail(span)
  highs = []
  for i in range(span):
    if i > 0:


      ticker.iloc[i]['Close']
      ticker.iloc[i-1]['Close']
      delta = float(ticker.iloc[i]['Close'])-float(ticker.iloc[i-1]['Close'])
      if i > 2:
        if delta_old > 0 and delta < 0:
          highs.append(float(ticker.iloc[i-1]['Close']))
          delta_old = delta
        else:
          delta_old = delta
      else:
        delta_old = delta
  if line == True:
    return np.array(highs), np.array(highs).mean()
  else:
    return np.array(highs)


def low_points(ticker, span, line):
  ticker = ticker.tail(span)
  lows = []
  for i in range(span):
    if i!= 0:
      delta = float(ticker.iloc[i]['Close'])-float(ticker.iloc[i-1]['Close'])
      if i > 2:
        if delta_old < 0 and delta > 0:
          lows.append(float(ticker.iloc[i-1]['Close']))
          delta_old = delta
        else:
          delta_old = delta
      else:
        delta_old = delta

  if line == True:
    return np.array(lows), np.array(lows).mean()
  else:
    return np.array(lows)

'''
def low_points(ticker, span, line):
  ticker = ticker.tail(span)
  local_minima = (ticker < ticker.shift(1)) & (ticker < ticker.shift(-1))
  ticker = ticker[local_minima]
  if line == True:
    return ticker, EMA(ticker, span)
  else:
    return ticker
'''
'''
np.random.seed(42)

# Generate a time series from 2023-01-01 for 100 days
dates = pd.date_range(start="2023-01-01", periods=100, freq="D")

# Generate random data for the 'close' column (e.g., random prices between 100 and 200)
close_prices = np.random.uniform(100, 200, size=len(dates))

# Create the DataFrame
df = pd.DataFrame({"Close": close_prices}, index=dates)

high, line = high_points(df, 50, line=True)
#print(high)
print(line)


low, line = low_points(df, 10, line=True)
print(line)
'''













'\nnp.random.seed(42)\n\n# Generate a time series from 2023-01-01 for 100 days\ndates = pd.date_range(start="2023-01-01", periods=100, freq="D")\n\n# Generate random data for the \'close\' column (e.g., random prices between 100 and 200)\nclose_prices = np.random.uniform(100, 200, size=len(dates))\n\n# Create the DataFrame\ndf = pd.DataFrame({"Close": close_prices}, index=dates)\n\nhigh, line = high_points(df, 50, line=True)\n#print(high)\nprint(line)\n\n\nlow, line = low_points(df, 10, line=True)\nprint(line)\n'

It is critically important not only to identify quality entries into the market but also quality exits. The exit defines the profit and loss. I can have middling entries and make a profit. If I have middling exits it does not matter how well I identify trends.

In this model I will focus on creating a realtively conservative strategy based on bricking out the fundamentals of my understanding before I begin to utilize more complex analytical tools and strategies.

In [ ]:
#3.
class Entry_Exit():
  def __init__(self, span, risk, lookback, threshold):
    self.span = span
    self.risk = risk
    self.lookback = lookback
    self.threshold = threshold


  def Entry(self, ticker, current, index):
    # plan your entrance

    # Calculate volatility which dictates size of position you will take
    volatility = Volatility(ticker, self.span)

    position_size = 1 + (-self.risk*volatility)

    ema = EMA(ticker, self.span*self.lookback)
    ticker.iloc[1]
    highs, high_ema = high_points(ticker, self.span, line=True)
    lows, low_ema = low_points(ticker, self.span, line=True)
    if(current <= 0.2):
      if(current > high_ema and current > ema):
        return position_size, current, high_ema, True

      elif(current < low_ema and current < ema):
        return position_size, current, low_ema, True

      else:
        return position_size, current, False


  def Exit(self, ticker, buy_price, current, long, index):
    delta = ticker.iloc[index]['Close'] - ticker.iloc[index-1]['Close']
    volatility = Volatility(ticker, self.span)
    # as volatility increases increase the loss threshold correspondingly
    self.threshold += volatility
    highs, high_ema = high_points(ticker, self.span, line=True)
    lows, low_ema = low_points(ticker, self.span, line=True)

    if (long == True):
      if(current < buy_price-(buy_price*self.threshold)):
        return True
      elif(delta < 0 and current < low_ema):
        return True
      else:
        return False


    else:
      if(current > buy_price+(buy_price*self.threshold)):
        return True
      elif(delta > 0 and current > high_ema):
        return True
      else:
        return False




In [ ]:
import itertools
#4. What parameter space will we deifine and what data will I use and how will it be packaged
# params: span, threshold, risk, lookback
param_space = [
    [20, 30, 50],
    [0.2, 0.4, 0.5, 0.6, 0.8],
    [0.2, 0.4, 0.5, 0.6, 0.8],
    [1,2,3,4,5]]


permutations = list(itertools.product(*param_space))

for i in permutations:
  Trader = Entry_Exit(i[0], i[1], i[2], i[3])
  for j in range(len(apple)):
    if (j > i[0]):
      #buy = Trader.Entry(apple.iloc[:j], float(apple.iloc[j]['Close']), j)
      sell = Trader.Exit(apple.iloc[:j], float(apple.iloc[j-i[0]]['Close']), float(apple.iloc[j]['Close']), True, j)
      if(sell == True):
  break








<ipython-input-101-f8b820d9097c>:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  buy = Trader.Entry(apple.iloc[:j], float(apple.iloc[j]['Close']), j)
<ipython-input-96-5ed94f9bb8e5>:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  delta = float(ticker.iloc[i]['Close'])-float(ticker.iloc[i-1]['Close'])
<ipython-input-96-5ed94f9bb8e5>:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  highs.append(float(ticker.iloc[i-1]['Close']))
<ipython-input-96-5ed94f9bb8e5>:46: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  delta = float(ticker.iloc[i]['Close'])-float(ticker.iloc[i-1]['Close'])
<ipython-input-

Streaming output truncated to the last 5000 lines.
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
7
8
9
10
11
12
1